<a href="https://colab.research.google.com/github/ArtyomShabunin/SMOPA-25/blob/main/lesson_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://prana-system.com/files/110/rds_color_full.png" alt="tot image" width="300"  align="center"/> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src="https://mpei.ru/AboutUniverse/OficialInfo/Attributes/PublishingImages/logo1.jpg" alt="mpei image" width="200" align="center"/>
<img src="https://mpei.ru/Structure/Universe/tanpe/structure/tfhe/PublishingImages/tot.png" alt="tot image" width="100"  align="center"/>

---

# **Системы машинного обучения и предиктивной аналитики в тепловой и возобновляемой энергетике**  

# ***Практические занятия***

---

# Занятие №14
# Разбор задач
# Глубокое обучение, pytorch
**21 мая 2025г.**

## Поиск аномальных данных

---
### Задача №1
#### Условие

Необходимо определить, когда установка работает в ненормальном режиме, используя параметры:

* Давление за компрессором
* Температура выхлопных газов
* Температура окружающей среды

Задачу решить с использованием модели автокодировщика с размером латентного пространста 2, путем заполнения пропущенных фрагментов кода (...).

In [ ]:
import numpy as np

# === Генерация данных ===
np.random.seed(0)
N_normal = 950
N_anomalous = 50

# Нормальные данные
P_comp = np.random.normal(10, 1, N_normal)
T_exhaust = 500 + 20 * (P_comp - 10) + np.random.normal(0, 5, N_normal)
T_ambient = np.random.normal(15, 3, N_normal)

# Аномалии
P_comp_anom = np.random.normal(6, 0.5, N_anomalous)
T_exhaust_anom = np.random.normal(700, 20, N_anomalous)  # перегрев
T_ambient_anom = np.random.normal(25, 2, N_anomalous)

# Объединение
X_normal = np.stack([P_comp, T_exhaust, T_ambient], axis=1)
X_anomalous = np.stack([P_comp_anom, T_exhaust_anom, T_ambient_anom], axis=1)
X_all = np.vstack([X_normal, X_anomalous])
y_all = np.array([0]*N_normal + [1]*N_anomalous)  # метки: 0 - норма, 1 - аномалия

#### Решение

In [ ]:
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.metrics import precision_score, recall_score, f1_score

# === Стандартизация ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_all)

# === Разделение ===
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_all, test_size=0.3, random_state=42)

# === Преобразование в тензоры ===
X_train_tensor = ...  # обучаемся только на норме
X_test_tensor = ...

In [ ]:
# === Автокодировщик ===
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            ...
        )
        self.decoder = nn.Sequential(
            ...
        )

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

model = Autoencoder()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = ...

# === Обучение ===
epochs = 100
for epoch in range(epochs):
    model.train()
    output = ...
    loss = ...
    ...
    ...
    ...
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

In [ ]:
# === Вычисление ошибки восстановления ===
model.eval()
with torch.no_grad():
    reconstructed = ...
    loss_per_sample = ...

# === Порог аномалии ===
threshold = np.percentile(loss_per_sample, 95)  # верхние 5% — потенциальные аномалии
predicted_anomaly = ...

# === Визуализация ===
# Цвета точек: синие — нормальные, красные — аномалии
colors = ['red' if y == 1 else 'blue' for y in y_test]

plt.figure(figsize=(10, 5))
plt.scatter(range(len(loss_per_sample)), loss_per_sample,
            c=colors, alpha=0.6, label='Samples')
plt.axhline(threshold, color='black', linestyle='--', label='Threshold')

# Легенда
normal_patch = mpatches.Patch(color='blue', label='Норма')
anomaly_patch = mpatches.Patch(color='red', label='Аномалия')
threshold_line = mpatches.Patch(color='black', label='Порог')

plt.legend(handles=[normal_patch, anomaly_patch, threshold_line])
plt.title("Ошибка реконструкции")
plt.xlabel("Индекс")
plt.ylabel("MSE")
plt.grid(True)
plt.tight_layout()
plt.show()

# Метрики
precision = ...
recall = ...
f1 = ...

print(f"Precision: {precision:.2f}")
print(f"Recall:    {recall:.2f}")
print(f"F1-score:  {f1:.2f}")

---
### Задача №2
#### Условие

Каждое наблюдение — это почасовой профиль потребления электроэнергии за сутки (24 значения).
Аномальные профили:

* Случайные всплески в ночное время
* Слишком ровный или хаотичный график

Решить задачу поиска аномалий с использованием модели автокодировщика с размером латентного пространства 3, путем заполнения пропущенных фрагментов кода (...).

In [ ]:
import numpy as np

# === Генерация данных ===
np.random.seed(0)
N_normal = 1000
N_anomaly = 100

# Нормальные профили: пик утром и вечером
def generate_normal():
    x = np.linspace(0, 2*np.pi, 24)
    profile = 2 + np.sin(x - np.pi/2) + 0.3 * np.random.randn(24)
    return profile

# Аномальные: шум или всплески ночью
def generate_anomaly():
    profile = np.random.normal(2, 0.3, 24)
    if np.random.rand() > 0.5:
        profile[0:6] += np.random.uniform(2, 4)  # ночной всплеск
    return profile

X_normal = np.array([generate_normal() for _ in range(N_normal)])
X_anomalous = np.array([generate_anomaly() for _ in range(N_anomaly)])
X_all = np.vstack([X_normal, X_anomalous])
y_all = np.array([0]*N_normal + [1]*N_anomaly)

#### Решение

In [ ]:
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.metrics import precision_score, recall_score, f1_score

# === Стандартизация ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_all)

# === Разделение ===
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_all, test_size=0.3, random_state=42)

# === Преобразование в тензоры ===
X_train_tensor = ...  # обучаемся только на норме
X_test_tensor = ...

In [ ]:
# === Автокодировщик ===
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            ...
            )
        self.decoder = nn.Sequential(
            ...
            )

    def forward(self, x):
        return ...

model = Autoencoder()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = ...

# === Обучение ===
for epoch in range(100):
    ...
    output = ...
    loss = ...
    ...
    ...
    ...
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

In [ ]:
# === Аномалия-оценка ===
model.eval()
with torch.no_grad():
    recon = ...
    loss_per_sample = ...

threshold = np.percentile(loss_per_sample, 95)
predicted_anomaly = ...

# === Визуализация ===
colors = ['red' if y == 1 else 'blue' for y in y_test]
plt.figure(figsize=(10, 5))
plt.scatter(range(len(loss_per_sample)), loss_per_sample, c=colors, alpha=0.6)
plt.axhline(threshold, color='black', linestyle='--', label='Threshold')
plt.legend(handles=[
    mpatches.Patch(color='blue', label='Норма'),
    mpatches.Patch(color='red', label='Аномалия'),
    mpatches.Patch(color='black', label='Порог')
])
plt.title("Ошибка реконструкции")
plt.xlabel("Индекс")
plt.ylabel("MSE")
plt.grid(True)
plt.tight_layout()
plt.show()

# Метрики
precision = ...
recall = ...
f1 = ...

print(f"Precision: {precision:.2f}")
print(f"Recall:    {recall:.2f}")
print(f"F1-score:  {f1:.2f}")

---
### Задача №3
#### Условие

Есть архив показаний 5 датчиков:

* Температура
* Вибрация
* Давление
* Сила тока
* Напряжение

Архив содержит следующие аномалии:

* Всплески в 1-2 параметрах
* Нестабильная работа (скачки)

Решить задачу поиска аномалий с использованием модели автокодировщика с размером латентного пространста 2, путем заполнения пропущенных фрагментов кода (...).

In [ ]:
import numpy as np

# === Генерация данных ===
def generate_normal_sensor():
    temp = np.random.normal(70, 2)
    vib = np.random.normal(0.02, 0.005)
    pres = np.random.normal(5, 0.3)
    current = np.random.normal(10, 1)
    voltage = np.random.normal(220, 3)
    return [temp, vib, pres, current, voltage]

def generate_anomaly_sensor():
    normal = generate_normal_sensor()
    i = np.random.choice(len(normal), size=np.random.randint(1, 3), replace=False)
    for idx in i:
        normal[idx] += np.random.uniform(5, 50)  # сильное отклонение
    return normal

N_normal = 1200
N_anomaly = 100

X_normal = np.array([generate_normal_sensor() for _ in range(N_normal)])
X_anomalous = np.array([generate_anomaly_sensor() for _ in range(N_anomaly)])
X_all = np.vstack([X_normal, X_anomalous])
y_all = np.array([0]*N_normal + [1]*N_anomaly)

#### Решение

In [ ]:
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.metrics import precision_score, recall_score, f1_score

# === Стандартизация ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_all)

# === Разделение ===
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_all, test_size=0.3, random_state=42)

# === Преобразование в тензоры ===
X_train_tensor = ...
X_test_tensor = ...

In [ ]:
# === Модель ===
class SensorAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = ...
        self.decoder = ...

    def forward(self, x):
        return ...

model = SensorAutoencoder()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = ...

# === Обучение ===
for epoch in range(100):
    ...
    out = ...
    loss = ...
    ...
    ...
    ...

In [ ]:
# === Результаты ===
model.eval()
with ...:
    recon = ...
    loss_per_sample = ...

threshold = np.percentile(loss_per_sample, 95)
predicted_anomaly = (loss_per_sample > threshold).astype(int)

# === Визуализация ===
colors = ['red' if y == 1 else 'blue' for y in y_test]
plt.figure(figsize=(10, 5))
plt.scatter(range(len(loss_per_sample)), loss_per_sample, c=colors, alpha=0.6)
plt.axhline(threshold, color='black', linestyle='--')
plt.legend(handles=[
    mpatches.Patch(color='blue', label='Норма'),
    mpatches.Patch(color='red', label='Аномалия'),
    mpatches.Patch(color='black', label='Порог')
])
plt.title("Ошибка реконструкции")
plt.xlabel("Индекс")
plt.ylabel("MSE")
plt.grid(True)
plt.tight_layout()
plt.show()

# Метрики
precision = ...
recall = ...
f1 = ...

print(f"Precision: {precision:.2f}")
print(f"Recall:    {recall:.2f}")
print(f"F1-score:  {f1:.2f}")

## Регрессия

---
### Задача №4
#### Условие
Предсказать температуру выхлопных газов газотурбинной установки на следующем временном шаге по:

* Положению топливного клапана
* Температуре окружающей среды
* Температуре за компрессором
* Давлению за компрессором

Задачу решить с помощью модели искусственной нейронной сети, путем заполнения пропущенных фрагментов кода (...).

In [ ]:
import numpy as np

# === Генерация синтетических данных ===
np.random.seed(0)
N = 1000

fuel_valve = np.random.uniform(0.2, 1.0, size=N)                # положение клапана
ambient_temp = np.random.normal(15, 5, size=N)                  # наружная температура
T_comp_out = 300 + 100 * fuel_valve + np.random.normal(0, 5, N) # T после компрессора
P_comp_out = 5 + 10 * fuel_valve + np.random.normal(0, 1, N)    # P после компрессора

# Целевая переменная: температура выхлопных газов
T_exhaust = (
    500 + 80 * fuel_valve - 0.5 * ambient_temp + 0.1 * T_comp_out
    + 2 * P_comp_out + np.random.normal(0, 5, N)
)

X = np.stack([fuel_valve, ambient_temp, T_comp_out, P_comp_out], axis=1)
y = T_exhaust

#### Решение

In [4]:
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# === Стандартизация ===
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y[:,np.newaxis])

# === Разделение на train/test ===
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.2, random_state=42)

# === Преобразование в тензоры ===
X_train_tensor = ...
y_train_tensor = ...
X_test_tensor = ...
y_test_tensor = ...

In [ ]:
# === Модель ===
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            ...
        )

    def forward(self, x):
        return self.model(x)

model = Regressor()
criterion = ...
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# === Обучение ===
epochs = 200
for epoch in range(epochs):
    ...
    output = ...
    loss = ...
    ...
    ...
    ...

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

In [ ]:
# === Предсказания и обратная стандартизация ===
model.eval()
with torch.no_grad():
    y_pred_scaled = ...
    y_pred = ...
    y_true = ...

# === График ===
plt.figure(figsize=(8, 5))
plt.plot(y_true, label='Истина')
plt.plot(y_pred, label='Предсказание')
plt.legend()
plt.title('Прогноз температуры выхлопных газов')
plt.xlabel('Индекс')
plt.ylabel('Температура выхлопных газов')
plt.grid(True)
plt.show()

# Расчет метрики R2
r2 = r2_score(y_pred, y_true)
print(f"R² score: {r2:.4f}")

---
### Задача №5
#### Условие
Необходимо спрогнозировать электрическую мощность газотурбинной установки (ГТУ) на основе:

* Давления за компрессором `P_comp_out`
* Температуры окружающего воздуха `T_ambient`
* Уровня подачи топлива `fuel_rate`

Задачу решить с помощью модели искусственной нейронной сети, путем заполнения пропущенных фрагментов кода (...).

In [ ]:
import numpy as np

# === Генерация данных ===
np.random.seed(1)
N = 1200

P_comp_out = np.random.normal(8.0, 1.0, size=N)                   # Давление за компрессором (бар)
T_ambient = np.random.normal(20, 6, size=N)                       # Температура воздуха (°C)
fuel_rate = np.random.uniform(0.3, 1.0, size=N)                   # Доля открытия топливной подачи (0-1)

# Целевая переменная: электрическая мощность (МВт)
power_output = (
    30 * fuel_rate + 5 * P_comp_out - 0.4 * T_ambient
    + np.random.normal(0, 2, size=N)
)

X = np.stack([P_comp_out, T_ambient, fuel_rate], axis=1)
y = power_output

#### Решение

In [ ]:
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# === Стандартизация ===
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

# === Train/Test split ===
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=1)

# === Преобразование в тензоры ===
X_train_tensor = ...
y_train_tensor = ...
X_test_tensor = ...
y_test_tensor = ...

In [ ]:
# === Модель ===
class PowerRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = ...
    def forward(self, x):
        return self.model(x)

model = ...
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = ...

# === Обучение ===
for epoch in range(150):
    ...
    y_pred = ...
    loss = ...
    ...
    ...
    ...
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

In [ ]:
# === Предсказания ===
...
with torch.no_grad():
    y_pred_test = ...
    y_pred_test_inv = ...
    y_true_test_inv = ...

# === Визуализация ===
plt.figure(figsize=(8, 5))
plt.plot(y_true_test_inv, label='Истина')
plt.plot(y_pred_test_inv, label='Предсказание')
plt.legend()
plt.title("Прогноз мощности ГТУ")
plt.xlabel("Индекс")
plt.ylabel("Мощность, МВт")
plt.grid(True)
plt.show()

# Расчет метрики R2
r2 = r2_score(y_pred_test_inv, y_true_test_inv)
print(f"R² score: {r2:.4f}")

---
### Задача №6
#### Условие

С течением времени КПД ГТУ деградирует из-за:

* Наработки (в часах)
* Загрязнения фильтров (0–1)
* Температуры окружающей среды

Нужно построить модель для прогнозирования текущего КПД установки.  
Задачу решить с помощью модели искусственной нейронной сети, путем заполнения пропущенных фрагментов кода (...).

In [ ]:
import numpy as np

# === Генерация данных ===
np.random.seed(2)
N = 1500

hours_run = np.random.uniform(0, 30000, size=N)                   # Наработка (часы)
filter_dirtiness = np.clip(np.random.beta(2, 5, size=N), 0, 1)    # Степень загрязнения
T_ambient = np.random.normal(18, 5, size=N)                       # Температура воздуха

# КПД (%)
efficiency = (
    42 - 0.0003 * hours_run - 2 * filter_dirtiness - 0.05 * T_ambient
    + np.random.normal(0, 0.5, size=N)
)

X = np.stack([hours_run, filter_dirtiness, T_ambient], axis=1)
y = efficiency

#### Решение

In [ ]:
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# === Стандартизация ===
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

# === Train/Test split ===
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# === Преобразование в тензоры ===
X_train_tensor = ...
y_train_tensor = ...
X_test_tensor = ...
y_test_tensor = ...

In [ ]:
# === Модель ===
class EfficiencyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = ...
    def forward(self, x):
        return self.model(x)

model = ...
optimizer = ...
criterion = ...

# === Обучение ===
for epoch in range(200):
    ...
    y_pred = ...
    loss = ...
    ...
    ...
    ...
    if epoch % 25 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

In [ ]:
# === Предсказания ===
model.eval()
with ...:
    y_pred_test = ...
    y_pred_test_inv = ...
    y_true_test_inv = ...

# === Визуализация ===
plt.figure(figsize=(8, 5))
plt.plot(y_true_test_inv, label='Истина')
plt.plot(y_pred_test_inv, label='Предсказание')
plt.legend()
plt.title("Прогноз КПД ГТУ")
plt.xlabel("Индекс")
plt.ylabel("КПД (%)")
plt.grid(True)
plt.show()

# Расчет метрики R2
r2 = r2_score(y_pred_test_inv, y_true_test_inv)
print(f"R² score: {r2:.4f}")

## Прогнозирование временных рядов

---
### Задача №7
#### Условие

Входные данные (на каждый час):

* Историческое потребление (на 24 часа назад)
* Время суток (от 0 до 23, как синус и косинус)
* День недели (0–6, как синус и косинус)

Цель:
Предсказать потребление на следующий час.  
Задачу решить с помощью модели искусственной нейронной сети, путем заполнения пропущенных фрагментов кода (...).

In [ ]:
import numpy as np

# === Генерация данных ==
np.random.seed(42)
n_hours = 3000

# Время: час суток и день недели
hour_of_day = np.arange(n_hours) % 24
day_of_week = (np.arange(n_hours) // 24) % 7

# Потребление: базовое + дневной цикл + влияние дня недели + шум
base = 50 + 10 * np.sin(2 * np.pi * hour_of_day / 24)
dow_effect = 5 * np.where(day_of_week < 5, 1.0, 0.7)  # рабочие и выходные
noise = np.random.normal(0, 2, n_hours)
consumption = base * dow_effect + noise  # финальное потребление

# Вспомогательная функция: кодировка времени
def encode_time(hour, dow):
    hour_sin = np.sin(2 * np.pi * hour / 24)
    hour_cos = np.cos(2 * np.pi * hour / 24)
    dow_sin = np.sin(2 * np.pi * dow / 7)
    dow_cos = np.cos(2 * np.pi * dow / 7)
    return np.array([hour_sin, hour_cos, dow_sin, dow_cos])  # shape (4,)

# Формирование выборки
window_size = 24
X, y = [], []

for i in range(n_hours - window_size - 1):
    past_consumption = consumption[i:i+window_size]  # shape (24,)
    time_feat = encode_time(hour_of_day[i+window_size], day_of_week[i+window_size])  # shape (4,)
    x = np.concatenate([past_consumption, time_feat])  # shape (28,)
    X.append(x)
    y.append(consumption[i + window_size])  # целевое значение на следующий час

X = np.array(X)
y = np.array(y).reshape(-1, 1)

#### Решение

In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# === Стандартизация ===
scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_x.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# === Преобразование в тензоры ===
X_tensor = ...
y_tensor = ...

# === Разделение на train/test ===
split_idx = int(0.8 * len(X_tensor))
X_train, X_test = ...
y_train, y_test = ...

dataset = TensorDataset(X_train, y_train)
train_loader = ...

In [ ]:
# === Модель ===
class FCNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            ...
        )

    def forward(self, x):
        return self.net(x)

model = ...


# === Обучение ===
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = ...

for epoch in range(20):
    ...
    total_loss = 0
    for batch_x, batch_y in ...:
        pred = ...
        loss = ...
        ...
        ...
        ...
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

In [ ]:
# === Предсказание и метрика ===
model.eval()
with torch.no_grad():
    y_pred_scaled = ...
    y_true_scaled = ...

    # Обратная стандартизация
    y_pred = ...
    y_true = ...

# R² score
r2 = r2_score(y_true, y_pred)
print(f"\nR² score on test set: {r2:.4f}")

# Визуализация
plt.figure(figsize=(10, 4))
plt.plot(y_true[:100], label="Истина")
plt.plot(y_pred[:100], label="Предсказание")
plt.title("Прогноз потребления офиса на следующий час")
plt.xlabel("Время (часы)")
plt.ylabel("Потребление")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

---
### Задача №8
#### Условие

Смоделируйте и предскажите почасовое потребление электроэнергии жилого дома на следующие 24 часа на основе предыдущих 168 часов (7 суток). Используйте простую RNN (или LSTM). Задачу решить путем заполнения пропущенных фрагментов кода (...).

In [ ]:
import numpy as np

# === Генерация данных ===

# Параметры генерации
np.random.seed(42)
n_days = 100
hours_per_day = 24
total_hours = n_days * hours_per_day

# Генерация сезонности и тренда
time = np.arange(total_hours)
daily_pattern = 2 * np.sin(2 * np.pi * time / 24)  # дневной цикл
weekly_pattern = 1.5 * np.sin(2 * np.pi * time / (24 * 7))  # недельный цикл
trend = 0.0005 * time  # небольшой рост
noise = np.random.normal(0, 0.3, size=total_hours)

# Итоговый временной ряд
consumption = 5 + daily_pattern + weekly_pattern + trend + noise

# Преобразуем в формат [samples, sequence_length]
window_size = 168
target_size = 24

X, y = [], []
for i in range(len(consumption) - window_size - target_size):
    X.append(consumption[i:i+window_size])
    y.append(consumption[i+window_size:i+window_size+target_size])

X = np.array(X)
y = np.array(y)

#### Решение

In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

# === Cтандартизация данных ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.reshape(-1, 1)).reshape(X.shape)
y_scaled = scaler.transform(y.reshape(-1, 1)).reshape(y.shape)

# === Преобразование в тензоры ===
X_tensor = ...
y_tensor = ...

In [ ]:
# === Модель ===
class EnergyLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=24):
        super().__init__()
        self.lstm = ...
        self.linear = ...

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # берем только последний скрытый вектор
        return self.linear(out)

model = ...

# === Обучение ===
dataset = TensorDataset(X_tensor.unsqueeze(-1), y_tensor)
loader = ...

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = ...

for epoch in range(10):
    for batch_X, batch_y in ...:
        pred = ...
        loss = ...
        ...
        ...
        ...
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

In [ ]:
# === Предсказание ===
# Предсказание на последнем фрагменте
with torch.no_grad():
    test_input = X_tensor[-1].unsqueeze(0).unsqueeze(-1)
    pred_scaled = ...
    pred = ...

plt.plot(np.arange(24), consumption[-24:], label='Истина')
plt.plot(np.arange(24), pred.flatten(), label='Предсказание')
plt.legend()
plt.title("Прогноз потребления энергии на 24 часа")
plt.show()

# Расчет метрики R2
y_true = consumption[-24:]
y_pred = pred.flatten()

r2 = r2_score(y_true, y_pred)
print(f"R² score: {r2:.4f}")

---
### Задача №9
#### Условие

На основе временных рядов:

* `ток` (текущая нагрузка трансформатора, A)
* `температура_внешняя` (температура воздуха, °C)

Необходимо предсказать:

* `температура_трансформатора` через 1 час.

Задачу решить с помощью модели искусственной нейронной сети, путем заполнения пропущенных фрагментов кода (...).

In [ ]:
import numpy as np

# === Генерация данных ===
np.random.seed(42)

n_hours = 3000  # 125 суток

time = np.arange(n_hours)
external_temp = 10 + 10 * np.sin(2 * np.pi * time / 24) + np.random.normal(0, 1, n_hours)
current = 100 + 40 * np.sin(2 * np.pi * time / 24 + np.pi/4) + np.random.normal(0, 5, n_hours)

# Температура трансформатора зависит от тока + внешней температуры + инерция (скользящее среднее)
temp_transformer = (
    0.5 * current + 0.3 * external_temp +
    0.2 * np.convolve(current, np.ones(5)/5, mode='same')
    + np.random.normal(0, 1, n_hours)
)

# Подготовка данных
window_size = 24  # используем 24 часа для прогноза на 1 час вперёд

X, y = [], []
for i in range(n_hours - window_size - 1):
    x_window = np.stack([
        current[i:i+window_size],
        external_temp[i:i+window_size]
    ], axis=1)
    X.append(x_window)
    y.append(temp_transformer[i+window_size])  # предсказание через 1 час

X = np.array(X)  # [samples, window, features]
y = np.array(y).reshape(-1, 1)

#### Решение

In [ ]:
import torch
from torch import nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader

# === Стандартизация ===
scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_reshaped = X.reshape(-1, 2)
X_scaled = scaler_x.fit_transform(X_reshaped).reshape(X.shape)
y_scaled = scaler_y.fit_transform(y)

# === Преобразование в тензоры ===
X_tensor = ...
y_tensor = ...

# === Разделение на train/test ===
split_idx = int(0.8 * len(X_tensor))
X_train, X_test = ...
y_train, y_test = ...

In [ ]:
# === Модель ===
class TransformerTempLSTM(nn.Module):
    def __init__(self, input_size=2, hidden_size=64, num_layers=1):
        super().__init__()
        self.lstm = ...
        self.linear = ...

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # последний временной шаг
        return self.linear(out)

model = ...

# === Обучение модели ===
dataset = TensorDataset(X_train, y_train)
train_loader = ...

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = ...

for epoch in range(20):
    for batch_x, batch_y in ...:
        pred = ...
        loss = ...
        ...
        ...
        ...
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

In [ ]:
# === Предсказание ===
with torch.no_grad():
    y_pred_scaled = model(X_test).numpy()
    y_true_scaled = y_test.numpy()

    # Обратная стандартизация
    y_pred = ...
    y_true = ...

# Метрика
r2 = r2_score(y_true, y_pred)
print(f"\nR² score on test set: {r2:.4f}")

# Визуализация
plt.figure(figsize=(10, 4))
plt.plot(y_true[:100], label='Истина')
plt.plot(y_pred[:100], label='предсказание')
plt.legend()
plt.title("Температура трансформатора — 1 час вперёд")
plt.show()

---
### Задача №10
#### Условие

Даны почасовые данные:

* Потребление зданий A, B, C
* Температура воздуха (°C)

Требуется:

* На основе данных за последние 24 часа предсказать **суммарное потребление** в районе через 1 час.

Задачу решить с помощью модели искусственной нейронной сети, путем заполнения пропущенных фрагментов кода (...).

In [ ]:
import numpy as np

# === Генерация данных ===
np.random.seed(42)

n_hours = 4000

time = np.arange(n_hours)

# Температура воздуха (дневной цикл + шум)
temp_outdoor = 10 + 10 * np.sin(2 * np.pi * time / 24) + np.random.normal(0, 1, n_hours)

# Потребление зданий: базовый уровень + зависимость от температуры и времени суток
def gen_building_usage(base, temp_weight, noise_level):
    usage = (
        base
        + temp_weight * (20 - temp_outdoor)  # холодно — больше потребление
        + 5 * np.sin(2 * np.pi * time / 24)  # дневной цикл
        + np.random.normal(0, noise_level, n_hours)
    )
    return usage

A = gen_building_usage(30, 0.8, 2)
B = gen_building_usage(40, 1.0, 3)
C = gen_building_usage(25, 0.5, 1.5)

# Целевая переменная: общее потребление
total_usage = A + B + C

# Подготовка данных
window_size = 24  # 24 часа истории
X, y = [], []

for i in range(n_hours - window_size - 1):
    x_window = np.stack([
        A[i:i+window_size],
        B[i:i+window_size],
        C[i:i+window_size],
        temp_outdoor[i:i+window_size]
    ], axis=1)
    X.append(x_window)
    y.append(total_usage[i+window_size])  # на 1 час вперёд

X = np.array(X)  # [samples, window_size, features]
y = np.array(y).reshape(-1, 1)

#### Решение

In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# === Стандартизация ===
scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_x.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
y_scaled = scaler_y.fit_transform(y)

# === Преобразование в тензоры ===
X_tensor = ...
y_tensor = ...

# Train/test split
split_idx = int(0.8 * len(X_tensor))
X_train, X_test = ...
y_train, y_test = ...

In [ ]:
# === Модель ===
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, d_model=64, nhead=4, num_layers=2):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.regressor = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.input_proj(x)  # [B, T, d_model]
        x = self.transformer(x)
        out = x[:, -1, :]  # последний временной шаг
        return self.regressor(out)

model = TimeSeriesTransformer(input_dim=4)

# === Обучение модели ===
dataset = TensorDataset(X_train, y_train)
train_loader = ...
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = ...

for epoch in range(20):
    ...
    total_loss = 0
    for batch_x, batch_y in train_loader:
        pred = ...
        loss = ...
        ...
        ...
        ...
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

In [ ]:
# === Предсказание ===
...
with ...:
    y_pred_scaled = ...
    y_true_scaled = ...

    y_pred = ...
    y_true = ...

# R² метрика
r2 = r2_score(y_true, y_pred)
print(f"\nR² score on test set: {r2:.4f}")

# Визуализация
plt.figure(figsize=(10, 4))
plt.plot(y_true[:100], label='Истина')
plt.plot(y_pred[:100], label='Предсказание')
plt.title("Суммарное потребление — прогноз на 1 час")
plt.legend()
plt.show()